<a href="https://colab.research.google.com/github/mayhd3/NSF-REU-2021/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

if not os.path.exists('Xtst.csv'):
  !gdown --id 1wQDp91LC0PBJvJRBTLw1aW51-UN5JOm8
if not os.path.exists('Ytr.csv'):
  !gdown --id 1ZhPVjXlOttrYRjSRm_IUBZj-Bokswzbn
if not os.path.exists('Ytst.csv'):
  !gdown --id 10LE7A7yhNkPlk8Zv3dmBjFYwMW9ZKy9_
if not os.path.exists('Xtr.csv'):
  !gdown --id 1IUcv_OcLX1YKUB1Xw_lYiBK6bnyLxikS
if not os.path.exists('cnn1d.zip'):
  !curl --remote-name -H 'Accept: application/vnd.github.v3.raw' --location 'https://github.com/mayhd3/NSF-REU-2021/raw/main/cnn1d.zip'

Downloading...
From: https://drive.google.com/uc?id=1wQDp91LC0PBJvJRBTLw1aW51-UN5JOm8
To: /content/Xtst.csv
308MB [00:05, 52.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZhPVjXlOttrYRjSRm_IUBZj-Bokswzbn
To: /content/Ytr.csv
25.7MB [00:00, 50.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=10LE7A7yhNkPlk8Zv3dmBjFYwMW9ZKy9_
To: /content/Ytst.csv
6.42MB [00:00, 30.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IUcv_OcLX1YKUB1Xw_lYiBK6bnyLxikS
To: /content/Xtr.csv
1.23GB [00:17, 70.1MB/s]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   134  100   134    0     0    401      0 --:--:-- --:--:-- --:--:--   401
100 1232k  100 1232k    0     0  1457k      0 --:--:-- --:--:-- --:--:-- 10.3M


In [55]:
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

Xtr = pd.read_csv("Xtr.csv", sep=',', header=None)
Xtst = pd.read_csv("Xtst.csv", sep=',', header=None)
Ytr = pd.read_csv("Ytr.csv", sep=',', header=None)
Ytst = pd.read_csv("Ytst.csv", sep=',', header=None)

def train_test_base_model(Xtr, Xtst, Ytr, Ytst):
  scaler = preprocessing.StandardScaler().fit(Xtr)
  xtr = scaler.transform(Xtr)  
  xtst = scaler.transform(Xtst)  

  xtr = xtr.reshape(xtr.shape[0],xtr.shape[1],1)
  xtst = xtst.reshape(xtst.shape[0],xtst.shape[1],1)

  print ("Number of training samples is: ",xtr.shape[0])
  print ("Number of test samples is: ",xtst.shape[0])

  num_classes = 2

  ytr = to_categorical(Ytr, num_classes)
  ytst = to_categorical(Ytst, num_classes)

  model = Sequential()
  model.add(Conv1D(32, kernel_size=7, activation='relu', input_shape=xtr.shape[1:]))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))    
  model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
  model.fit(xtr, ytr, epochs=150, verbose=2, batch_size=128,validation_split=0.3)  #learning_rte
  print("Evaluation of best performing model:")
  model.evaluate(xtst, ytst)
  return model

if not os.path.exists('cnn1d.zip'):
  model = train_test_base_model(Xtr,Xtst,Ytr,Ytst)
  model.save('cnn1d')
  !zip -r cnn1d.zip cnn1d
else:
  if not os.path.exists('cnn1d'):
    !unzip cnn1d.zip
  model = load_model('cnn1d')

In [ ]:
import numpy as np

def percentiles(x,y,p):
# labels = np.argmax(y, axis=1)
  bisect = [ np.compress(y == label, x, axis=0) for label in [0.0,1.0] ]
  print(x.shape)
  totals = [ np.sum(part, axis=1) for part in bisect ]
  print([np.percentile(total, p) for total in totals])
  return (y == 0.) & (np.sum(x, axis=1) < np.percentile(totals[0],p))
    # [bisect[i][totals[i] < np.percentile(totals[i], p)] for i in [0,1]]

Ytr_low = percentiles(Xtr.to_numpy(),Ytr.to_numpy().flatten(),10).astype(int)
Ytst_low = percentiles(Xtst.to_numpy(),Ytst.to_numpy().flatten(),10).astype(int)

discriminator = train_test_base_model(Xtr, Xtst, Ytr_low, Ytst_low)

(1029085, 48)
[5.958668296039206, 2.203336569764192]
(256883, 48)
[6.358757727843121, 2.561238887802642]
Number of training samples is:  1029085
Number of test samples is:  256883
Epoch 1/150


In [8]:
try:
  from art.attacks.evasion import FastGradientMethod, Wasserstein
  from art.estimators.classification import TensorFlowV2Classifier
except ImportError:
  import sys
  !{sys.executable} -m pip install adversarial-robustness-toolbox[Keras]
  from art.attacks.evasion import FastGradientMethod, Wasserstein
  from art.estimators.classification import TensorFlowV2Classifier
import tensorflow as tf

# Train the ART classifier

classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=xtr.shape,
    loss_object=tf.keras.losses.CategoricalCrossentropy(
      from_logits=False,
#      label_smoothing=0,
#      axis=-1,
#      reduction="auto",
      name="categorical_crossentropy"
  )
)
predictions = classifier.predict(xtst)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(ytst, axis=1)) / len(ytst)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

# Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.2)
xtst_adv = attack.generate(x=xtst)

# Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(xtst_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(ytst, axis=1)) / len(ytst)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

     |████████████████████████████████| 1.1 MB 4.0 MB/s 
     |████████████████████████████████| 3.4 MB 20.0 MB/s 
     |████████████████████████████████| 25.3 MB 82 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
Accuracy on benign test examples: 93.67416294577687%
Accuracy on adversarial test examples: 66.1472343440399%
